# 东方财富网国内新闻数据分析器
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
### 此程序将对指定股票数据进行分析！

# 参考资料
* [1] [Basics of SFrame](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [Remove Multiple Substring from String](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)

# 导入库

In [16]:
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate as tc
import csv
from bs4 import BeautifulSoup

# 准备阶段
* 1. 定义用于搜寻的板块
* 2. 创建一个包含25个链接的搜寻目录
* 3. 创建一个包含20个Class的搜寻目录

In [17]:
# 新闻板块(同时也会加入热门新闻的数据)
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录板块

# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个Class的目录！')
# print(class_list)  # Debug

成功创建包含 25 个页面链接的目录！
成功创建包含 20 个class的目录！


# 获得所有文章的链接

In [20]:
# 初始化urllist
urllist = []
year = '2018'  # Separate parameter
counter = 1

# 创造urllist
print('-----------------获取所有文章超链接程序-----------------')
print('分析任务开始！')
for url in page_list:
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    for each_url in soup.find_all('a', href=True):
        if 'http://finance.eastmoney.com/news/' in each_url['href']:
            if year in each_url['href']:
                urllist.append(each_url['href'])   
    print('第' + str(counter) + '页分析完成！现在一共获得了' + str(len(urllist)) + '篇文章的链接！')
    counter += 1

urllist = list(set(urllist))
print('全部分析完成！正在查重...')
print('任务完成！共获得了' + str(len(urllist)) + '篇文章的链接！')
print('---------------------------------------------')

-----------------获取所有文章超链接程序-----------------
分析任务开始！
第1页分析完成！现在一共获得了37篇文章的链接！
第2页分析完成！现在一共获得了73篇文章的链接！
第3页分析完成！现在一共获得了108篇文章的链接！
第4页分析完成！现在一共获得了146篇文章的链接！
第5页分析完成！现在一共获得了183篇文章的链接！
第6页分析完成！现在一共获得了223篇文章的链接！
第7页分析完成！现在一共获得了262篇文章的链接！
第8页分析完成！现在一共获得了299篇文章的链接！
第9页分析完成！现在一共获得了335篇文章的链接！
第10页分析完成！现在一共获得了371篇文章的链接！
第11页分析完成！现在一共获得了406篇文章的链接！
第12页分析完成！现在一共获得了441篇文章的链接！
第13页分析完成！现在一共获得了478篇文章的链接！
第14页分析完成！现在一共获得了518篇文章的链接！
第15页分析完成！现在一共获得了556篇文章的链接！
第16页分析完成！现在一共获得了592篇文章的链接！
第17页分析完成！现在一共获得了627篇文章的链接！
第18页分析完成！现在一共获得了664篇文章的链接！
第19页分析完成！现在一共获得了699篇文章的链接！
第20页分析完成！现在一共获得了734篇文章的链接！
第21页分析完成！现在一共获得了771篇文章的链接！
第22页分析完成！现在一共获得了816篇文章的链接！
第23页分析完成！现在一共获得了852篇文章的链接！
第24页分析完成！现在一共获得了887篇文章的链接！
第25页分析完成！现在一共获得了922篇文章的链接！
全部分析完成！正在查重...
任务完成！共获得了515篇文章的链接！
---------------------------------------------


# 将各个网页的资料抓取进SFrame
* 1. 新建用于抓取的函数
* 2. 实施抓取

In [114]:
# 清洗数据的词库
banned_info = ['责任编辑','原标题']
banned_words = ['摘要\n', '\n', '\r','\u3000','（中国新闻网）','（来源：第一财经）']

In [117]:
# 抓取新闻URL函数
def collectNews(news, url, counter):
    # 准备工作
    req = urllib.request.Request(url)
    print('请求链接连接成功!')
    response = urllib.request.urlopen(req)
    print('收到反馈信号！')
    html = response.read()
    print('HTML生成完毕！')
    soup = BeautifulSoup(html, "lxml")
    print('SOUP创建完成！')
    
    # 获得文章的发表时间 
    time = soup.find(class_="time").get_text()
    print('时间获取完毕！')
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text() 
    
    # 清洗数据    
    for banned_information in banned_info:
        if banned_information in content:
            content = content[0:re.search(banned_information, content).span()[0]-1].strip()

    for banned_word in banned_words:
        content = content.replace(banned_word, '')  # 效率低 但是可忽略不计
    print('文章内容获取完成！')

    
    # 获得相关的股票单
    related_stocks = []
    for each in soup.find_all(class_='keytip'): 
        related_stocks.append(each.get_text())
    print('相关内容单获取完成！')         
    
    # 写入SFrame
    temp_sframe = tc.SFrame({'year':[str(time[0:4])],
                             'month':[str(time[5:7])],
                             'day':[str(time[8:10])],
                             'title':[soup.find('h1').get_text()],
                             'contents':[content], 
                             'related':[related_stocks]})
    news = news.append(temp_sframe)
    print('SFrame写入完成!')
    # print(news)  # Debug
    
    
    # 释放内存
    del(req, response, html, soup, time, content, related_stocks)
    
    # 刷新计数 
    counter += 1
    print('计数刷新完成！')
    print('页面数据获取完毕！')
    return news

In [ ]:
# 初始化计数器
counter = 1

# 获取总任务数
total = len(urllist)

# 创建SFrame
news = tc.SFrame({'year':['0000'],'month':['00'],'day':['00'],'title':['Null Title'],'contents':['Null Contents'],'related':[['Null', 'Null']]})

# 下载数据
for each_article in urllist:
    print('=============================================')
    print('正在获取第' + str(counter) + '篇文章.')
    print('---------------------------------------------')
    news = collectNews(news, each_article, counter) 
    counter += 1
    
print('获取完毕！共获取了' + str(total) + '篇文章!')

正在获取第1篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第2篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第3篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第4篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第5篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第6篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第7篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
H

收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第43篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第44篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第45篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第46篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第47篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计数刷新完成！
页面数据获取完毕！
正在获取第48篇文章.
---------------------------------------------
请求链接连接成功!
收到反馈信号！
HTML生成完毕！
SOUP创建完成！
时间获取完毕！
文章内容获取完成！
相关内容单获取完成！
SFrame写入完成!
计

In [92]:
news[3]

{'contents': '【个人所得税法修正案(草案)向社会公开征集意见】中国人大网公布《中华人民共和国个人所得税法修正案(草案)》全文，并向社会公开征集意见，征求意见截止日期为7月28日。《中华人民共和国个人所得税法修正案(草案)》29日在中国人大网公布，向社会征求意见，征求意见截止日期2018年7月28日。根据草案，个税起征点拟由每月3500元提至每月5000元(每年6万元)，还首次增加子女教育支出、继续教育支出、大病医疗支出、住房贷款利息和住房租金等专项附加扣除。对于此次修法，你有啥意见？【相关报道】个税时间表：5000元起征点新政有望10月1日起施行6月29日，中国人大网公布了《中华人民共和国个人所得税法修正案(草案)》(下称个税修正案)全文，向社会公众征求意见，截止日期为7月28日。第一财经记者注意到，个税修正案提出在2019年1月1日施行，但在施行前，自2018年10月1日至2018年12月31日，纳税人的工资、薪金所得，先行以每月收入额减除费用五千元后的余额为应纳税所得额，并按照新的税率表来计算纳税款。这意味着个税减税红包有望在10月1日起施行。这次个税改革一大亮点是，将工资、薪金所得，劳务报酬所得，稿酬所得，特许权使用费所得等4项劳动性所得(下称“综合所得”)纳入综合征税范围，适用统一的超额累进税率。但此前官方并未详细披露综合所得收入适用超额累进税率级距。包括工资、薪金等4项综合所得的税率范围首次对外公布。此次个税修正案明确了这一问题，值得注意的是这次税率表中的应纳税所得额不再是以前按月计征，而是按年。而且应纳税所得额扣除标准，除了此前公布的6万元(即每个月5000元)外，还包括目前已经施行的“三险一金”(基本养老保险、基本医疗保险、失业保险等社会保险费和住房公积金)专项扣除，以及此次新增的子女教育、继续教育、大病医疗、住房贷款利息和住房租金等支出的专项附加扣除。老百姓在将自己的税前收入扣除上述各类项目支出后的收入即应纳税所得额，适用以下的新税率和级距。根据个税修正案公布的综合所得个人所得税税率表：全年应纳税所得额不超过3.6万元的，适用最低档的3%税率。超过3.6万元至14.4万元的部分，适用10%税率。超过14.4万元至30万元的部分，适用20%税率。超过30万元至42万元的部分，适用25%的税率。超过42万元至66万元的部分适用30%的税率。

# 处理数据
* 1.去除换行符
* 2.去除空白